This notebook runs a pipeline with the following steps:
* Create a test Visibility dataset with two components and no beam.
* Write to disk in MSv2 format.
* Read MSv2 data into Visibility dataset.$^{*}$
* Predict model visibilities.$^{*}$
* Do bandpass calibration.$^{*}$
* Check for convergence.$^{*}$

\* xarray dataset map_blocks() is used to distribute frequency sub-bands across dask tasks.

The pipeline is called once with the default internal dask cluster,
then again with an externally generated cluster.

In [1]:
from time import perf_counter

from dask.distributed import LocalCluster
from ska_sdp_instrumental_calibration.processing_tasks.lsm import Component
from ska_sdp_instrumental_calibration.workflow.pipelines import (
    bandpass_calibration,
)

<frozen abc>:106: FutureWarning: xarray subclass Visibility should explicitly define __slots__


In [2]:
# Simple bandpass calibration with internally generated cluster
t0 = perf_counter()
bandpass_calibration.run(
    {
        "lsm": [
            Component(
                name="comp1", RAdeg=20.0, DEdeg=-30.0, flux=1.0, alpha=0.0
            ),
            Component(
                name="comp2", RAdeg=19.5, DEdeg=-29.9, flux=0.1, alpha=0.0
            ),
        ],
        "beam_type": "none",
        "ms_name": "demo.ms",
        "hdf5_name": "demo.hdf5",
    }
)
print(f"pipeline ran in {perf_counter() - t0:.1f} seconds")

2024-12-18 12:37:35,447 - workflow.pipeline_config - INFO - Simulating demo MSv2 input
2024-12-18 12:37:35,448 - workflow.pipeline_config - INFO - Predicting visibilities without a beam
2024-12-18 12:37:35,449 - workflow.pipeline_config - INFO - Generating a demo MSv2 Measurement Set demo.ms
2024-12-18 12:37:35,466 - workflow.utils - INFO - Using AA1-Low with 20 stations
2024-12-18 12:37:36,030 - processing_tasks.predict - INFO - No beam model used in predict
2024-12-18 12:37:36,199 - workflow.utils - INFO - Applying direction-independent gain corruptions
2024-12-18 12:37:36,720 - pipeline.bandpass_calibration - INFO - Starting pipeline with 16-channel chunks
2024-12-18 12:37:36,721 - pipeline.bandpass_calibration - INFO - No dask cluster supplied. Using LocalCluster
2024-12-18 12:37:37,613 - pipeline.bandpass_calibration - INFO - Will read from demo.ms in 16-channel chunks
2024-12-18 12:37:37,747 - pipeline.bandpass_calibration - INFO - Setting vis predict in 16-channel chunks
2024-12

pipeline ran in 5.2 seconds


In [3]:
# Simple bandpass calibration with externally generated cluster
t0 = perf_counter()
cluster = LocalCluster()
print(f"Sending scheduler address {cluster.scheduler_address}")
bandpass_calibration.run(
    {
        "dask_scheduler_address": cluster.scheduler_address,
        "lsm": [
            Component(
                name="comp1", RAdeg=20.0, DEdeg=-30.0, flux=1.0, alpha=0.0
            ),
            Component(
                name="comp2", RAdeg=19.5, DEdeg=-29.9, flux=0.1, alpha=0.0
            ),
        ],
        "beam_type": "none",
        "ms_name": "demo.ms",
        "hdf5_name": "demo.hdf5",
    }
)
print(f"pipeline ran in {perf_counter() - t0:.1f} seconds")

2024-12-18 12:37:41,447 - workflow.pipeline_config - INFO - Simulating demo MSv2 input
2024-12-18 12:37:41,448 - workflow.pipeline_config - INFO - Predicting visibilities without a beam
2024-12-18 12:37:41,449 - workflow.pipeline_config - INFO - Generating a demo MSv2 Measurement Set demo.ms
2024-12-18 12:37:41,472 - workflow.utils - INFO - Using AA1-Low with 20 stations
2024-12-18 12:37:41,509 - processing_tasks.predict - INFO - No beam model used in predict
2024-12-18 12:37:41,618 - workflow.utils - INFO - Applying direction-independent gain corruptions


Sending scheduler address tcp://127.0.0.1:39897


2024-12-18 12:37:42,226 - pipeline.bandpass_calibration - INFO - Starting pipeline with 16-channel chunks
2024-12-18 12:37:42,228 - pipeline.bandpass_calibration - INFO - Using existing dask cluster tcp://127.0.0.1:39897
2024-12-18 12:37:42,250 - pipeline.bandpass_calibration - INFO - Will read from demo.ms in 16-channel chunks
2024-12-18 12:37:42,335 - pipeline.bandpass_calibration - INFO - Setting vis predict in 16-channel chunks
2024-12-18 12:37:42,379 - pipeline.bandpass_calibration - INFO - Setting calibration in 16-channel chunks
2024-12-18 12:37:42,457 - pipeline.bandpass_calibration - INFO - Running graph and returning calibration solutions
<frozen abc>:106: FutureWarning: xarray subclass Visibility should explicitly define __slots__
<frozen abc>:106: FutureWarning: xarray subclass Visibility should explicitly define __slots__
<frozen abc>:106: FutureWarning: xarray subclass Visibility should explicitly define __slots__
<frozen abc>:106: FutureWarning: xarray subclass Visibilit

pipeline ran in 3.6 seconds
